In [9]:
import numpy as np
from music21 import converter, instrument, note, chord, meter, duration, stream
import music21
import os
from tqdm import tqdm
from music21 import common

def insert_note(arr, note, offset_16th, single_note_length): 
    idx = int(offset_16th + (note.beat - 1) * single_note_length)
    if idx >= len(arr):
        return
    arr[int(note.pitch.midi), idx] = True

    note_off = idx + int(note.duration.quarterLength * 4) - 1
    if note_off >= arr.shape[1]:
        note_off = arr.shape[1] - 1
    arr[-1, note_off] = True

def encode_midi(measureMap, t=32, measure_length=16, single_note_length = 4, header=None):
    data = np.zeros((130, t), dtype=np.bool)
    measure_count = 0
    fill_header = True

    measure_keys = measureMap.keys()
    measure_keys = sorted(measure_keys)

    #for i in range(0, t, measure_length):
    for key in measure_keys:
        i = key * single_note_length
        if i >= t:
            break
        try:
            for element in measureMap[key][0]:
                #print(element)
                if isinstance(element, note.Note):
                    fill_header = False
                    insert_note(data, element, i, single_note_length)
                elif fill_header and header != None:
                    print(type(element))
                    header.append(element)
        except:
            print(measure_length)
            print(single_note_length)
            print(float(i / single_note_length))
            print(measureMap)

            """elif isinstance(element, music21.clef.TrebleClef) or isinstance(element, music21.clef.BassClef):
                clf = element
            elif isinstance(element, music21.tempo.MetronomeMark):
                tempo = element
            elif isinstance(element, music21.key.Key):
                song_key = element
            elif isinstance(element, music21.instrument.Instrument):
                inst = element
            elif isinstance(element, music21.meter.TimeSignature):
                ts = element
            else:
                print(type(element))"""
    
    return data

def decode_midi(data, header=None):
    strm = stream.Stream()
    
    for element in header[1:]: # TODO: right now I'm skipping the instrument because it adds like 10 empty measures for some reason
        strm.append(element)

    current_note = None
    current_length = 0.25

    for t in range(data.shape[1]):
        nothing = True

        for p, val in enumerate(data[:,t]):
            if val:
                if p < 128:
                    current_note = note.Note()
                    current_note.pitch.midi = p
                    current_length = 0.25

                    nothing = False
                elif p == 128:
                    pass #TODO: something with rests
                else:
                    if nothing:
                        current_length += 0.25
                    current_note.duration = duration.Duration(current_length)
                    #print('{}\t{}'.format(current_note.pitch, current_note.duration))
                    strm.append(current_note)
                    current_note = None
                    nothing = False
        
        if nothing and current_note: # if there was no data in this time step and there is a current note increase it's length by 1 16th
            current_length += 0.25
    
    return strm

def load_single(midi, measures=16):
    #midi = converter.parse(path)
    ts = midi.getTimeSignatures()[0]
    num = ts.numerator
    denom = ts.denominator

    measure_length = None # length of measure in 16ths
    single_note_length = None # length of a single note in 16ths

    if denom / 4 == 1:
        measure_length = num * 4 # quarter notes
        single_note_length = 4
    elif denom / 4 == 2:
        measure_length = num * 2 # eigth notes
        single_note_length = 2
    elif denom / 4 == 4:
        measure_length = num # 16ths
        single_note_length = 1
    elif denom / 4 == 0.5:
        measure_length = num * 8
        single_note_length = 8
    else:
        print(denom)


    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        #notes_to_parse = parts.parts[0].recurse()
        parts.parts[0].makeMeasures(inPlace=True) # TODO: select specific instrument name
        measureMap = parts.parts[0].measureOffsetMap()
    else: # file has notes in a flat structure
        #notes_to_parse = midi.flat.notes
        measureMap = midi.flat.measureOffsetMap()

    """ print(measureMap)
    return """
    return encode_midi(measureMap, t=measures*16, measure_length=measure_length, single_note_length = single_note_length) # TODO: fix t=measures*16

def parse(filename):
    return converter.parse(filename)
    """ try:
        return converter.parse(filename)
    except:
        return None """

def load_data(folder):
    filenames = [folder + f for f in os.listdir(folder)]
    results = common.runParallel(filenames, parse, updateFunction=True)
    #results = [parse(f) for f in filenames]
    #result = Parallel(n_jobs=4, backend="threading", verbose=1)(delayed(load_single)(folder + f, measures=16) for f in filenames)
    data = []
    for midi in tqdm(results):
        if midi:
            data.append(load_single(midi))
    
    return np.array(data)



In [13]:

train_x = load_data('data/train/')


Done 0 tasks of 1534
Done 9 tasks of 1534
Done 18 tasks of 1534
Done 27 tasks of 1534
Done 36 tasks of 1534
Done 45 tasks of 1534
Done 54 tasks of 1534
Done 63 tasks of 1534
Done 72 tasks of 1534
Done 81 tasks of 1534
Done 90 tasks of 1534
Done 99 tasks of 1534
Done 108 tasks of 1534
Done 117 tasks of 1534
Done 126 tasks of 1534
Done 135 tasks of 1534
Done 144 tasks of 1534
Done 153 tasks of 1534
Done 162 tasks of 1534
Done 171 tasks of 1534
Done 180 tasks of 1534
Done 189 tasks of 1534
Done 198 tasks of 1534
Done 207 tasks of 1534
Done 216 tasks of 1534
Done 225 tasks of 1534
Done 234 tasks of 1534
Done 243 tasks of 1534
Done 252 tasks of 1534
Done 261 tasks of 1534
Done 270 tasks of 1534
Done 279 tasks of 1534
Done 288 tasks of 1534
Done 297 tasks of 1534
Done 306 tasks of 1534
Done 315 tasks of 1534
Done 324 tasks of 1534
Done 333 tasks of 1534
Done 342 tasks of 1534
Done 351 tasks of 1534
Done 360 tasks of 1534
Done 369 tasks of 1534
Done 378 tasks of 1534
Done 387 tasks of 1534
Do

In [12]:
np.save('data/validation_data.npy', train_x)

In [209]:
header = []
measures = 16
data = encode_midi(measureMap, t=measures*16, header=header)
strm = decode_midi(data, header)
strm.ticksPerQuarterNote = 1024
strm.makeRests(fillGaps=True, inPlace=True)
#strm.show('text')
strm.write('midi', fp='data/Sin_City_decoded.mid')

<class 'music21.instrument.ElectricBass'>
<class 'music21.clef.Bass8vbClef'>
<class 'music21.tempo.MetronomeMark'>
<class 'music21.key.Key'>
<class 'music21.meter.TimeSignature'>


'data/Sin_City_decoded.mid'